<a href="https://colab.research.google.com/github/kt-chan/cuda-demo/blob/main/cuda_cplusplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用Google Colab寫C++程式並運行

Create sample c++ code with %%writefile filename.cpp

In [ ]:
%%writefile demo.cpp

#include <iostream>
using namespace std;
int main()
{
    string text = "world2";
    cout << "hello, " + text;
}

Overwriting demo.cpp


Compile the code with %%shell command

In [ ]:
%%shell

g++ demo.cpp -o demo

Execution by just run it, with %%shell command.

In [ ]:
%%shell
./demo

Hello World!


# 配置 CUDA Environment

In [ ]:
# check nvidia card info
!nvidia-smi

Thu May 23 04:01:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# check os info
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [ ]:
#get current working directory
!pwd

/content


Remote all legacy cuda framework, and update to latest version Go here: https://developer.nvidia.com/cuda-downloads

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/12.5.0/local_installers/cuda-repo-ubuntu2204-12-5-local_12.5.0-555.42.02-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2204-12-5-local_12.5.0-555.42.02-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2204-12-5-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda-toolkit-12-5

--2024-05-22 09:49:35--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2204.pin’

cuda-ubuntu2204.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-05-22 09:49:36 (4.09 MB/s) - ‘cuda-ubuntu2204.pin’ saved [190/190]

--2024-05-22 09:49:36--  https://developer.download.nvidia.com/compute/cuda/12.5.0/local_installers/cuda-repo-ubuntu2204-12-5-local_12.5.0-555.42.02-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [ ]:
#After refresh the cuda framework, check version info

!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# Setup Google Colab for cuda c++, rerun this for new session.

set your runtime to cuda by click "runtime" -> "change runtime type" in above toolbar, and select T4 GPU.

First, you have to install nvcc plugin for cuda compiler

In [ ]:
!pip install nvcc4jupyter

then, Load the plugin



In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpmtce1qif".


In [ ]:
# check nvidia card info
!nvidia-smi

Thu May 23 08:36:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Demo Code - Simple Loop

Simple 1 layer loop demo

In [ ]:
%%cuda_group_save -n demo.cu -g share

#include <cstdio>
#include <iostream>
#include <chrono>
#include <thread>
#include <cuda_runtime.h>
#include <atomic>

#define Debug false


using namespace std;

atomic<bool> running(true);


void checkGpuMemory() {
    size_t free, total;
    cudaError_t status;
    status = cudaMemGetInfo(&free, &total);
    if (status != cudaSuccess) {
        std::cerr << "Error checking GPU memory: " << cudaGetErrorString(status) << std::endl;
        return;
    }

    // Convert bytes to megabytes
    double freeMB = static_cast<double>(free) / (1024 * 1024);
    double totalMB = static_cast<double>(total) / (1024 * 1024);
    cout << "\n";
    cout << "Free Memory: " << freeMB << " MB" << std::endl;
    cout << "Total Memory: " << totalMB << " MB" << std::endl;
}


void rand(int *a, int n){
  for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		if(Debug) cout << a[i] << "\t";
	}
  if(Debug) cout << "\n"; else cout << a[1] << "\n";
}


void cpuadd (int *a, int *b, int *c, int n ) {
  for (int i=0; i<n; i++)
    {
        c[i] = a[i] + b[i];
    }
}

__global__ void vecadd_kernel (int *a, int *b, int *c, int n ) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n ) {  // Protect against out of bounds error
      c[i] = a[i] + b[i];
  }
}


void gpuadd (int *a, int *b, int *c, int n ) {
  int *d_a , *d_b , *d_c;
  cudaMalloc(&d_a, n*sizeof(int));
  cudaMalloc(&d_b, n*sizeof(int));
  cudaMalloc(&d_c, n*sizeof(int));
  cudaMemcpy(d_a, a, n*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

  // define kernel call
  int grids = max(1, (n+255)/256/256);
  vecadd_kernel<<<grids,256>>>(d_a, d_b, d_c, n);

  // Sync Device to Host
  cudaDeviceSynchronize();

  cudaMemcpy(c, d_c, n*sizeof(int), cudaMemcpyDeviceToHost);
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
}

void checkGPU(){
    const char* gpu_env = std::getenv("COLAB_GPU");
    if (gpu_env && std::atoi(gpu_env) > 0) {
        cout << "A GPU is connected." << endl;
    } else {
        cout << "No accelerator is connected." << endl;
    }
}

int main(void) {

  checkGPU();
  checkGpuMemory();

  int N=1<<28;
  cout << "n:\t" << N << "\n\n";

  int *a, *b, *c;
  //a = (int*)malloc(N*sizeof (int));
  //b = (int*)malloc(N*sizeof (int));
  //c = (int*)malloc(N*sizeof (int));
  cudaMallocHost(&a, N*sizeof(int));
  cudaMallocHost(&b, N*sizeof(int));
  cudaMallocHost(&c, N*sizeof(int));

  cout << "a:\t";
  rand(a, N);
  cout << "b:\t";
  rand(b, N);

  cout << "\n\n@CPU, summing value... \n";

  clock_t t;

  // calling cpu
  t=clock();//start time
  cpuadd(a,b,c,N);
  t = clock() - t;//total time = end time - start time

  if(Debug) {
    for (int i = 0; i < N; i++) {
      cout << c[i] << "\t";
    }
    cout << "\n";
  } else {
      cout << c[1] << "\n";
  }
  printf ("CPU Avg time = %lf ms.\n",((((float)t)/CLOCKS_PER_SEC)*1000));

  // calling gpu
  cout << "\n\n@GPU, summing value... \n";

  t=clock();//start time
  gpuadd(a,b,c,N);
  t = clock() - t;//total time = end time - start time


  if(Debug) {
    for (int i = 0; i < N; i++) {
      cout << c[i] << "\t";
    }
    cout << "\n";
  } else {
      cout << c[1] << "\n";
  }
  printf ("GPU Avg time = %lf ms.\n",((((float)t)/CLOCKS_PER_SEC)*1000));

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}

In [ ]:
%cuda_group_run -g "share"

A GPU is connected.

Free Memory: 14999.1 MB
Total Memory: 15102.1 MB
n:	268435456

a:	41624518
b:	158492008


@CPU, summing value... 
200116526
CPU Avg time = 897.617981 ms.


@GPU, summing value... 
200116526
GPU Avg time = 269.792999 ms.



# Demo Code - Complex nested loop

Complex N layer loop demo

CPU implementation

In [ ]:
%%writefile cpu-demo.cpp

#include <cstdio>
#include <iostream>

#define Debug false


using namespace std;


void rand(int *a, int n){
  for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		if(Debug) cout << a[i] << "\t";
	}
  if(Debug) cout << "\n";
}


void cpucal (int *a, int *b, int *c, int n ) {
  // calculate the average of each of a / each of b
  for (int i=0; i<n; i++){
    for (int j=0; j<n; j++) {
       if(b[j] != 0) {
          average.add(a[i]/b[j]);
       }
       else {
        average.add(0);
       }
    }
    c[i] = average.getAverage();
  }
}

void checkGPU(){
    const char* gpu_env = std::getenv("COLAB_GPU");
    if (gpu_env && std::atoi(gpu_env) > 0) {
        cout << "A GPU is connected." << endl;
    } else {
        cout << "No accelerator is connected." << endl;
    }
}

int main(void) {

  checkGPU();

  int N=1<<15;
  cout << "n:\t" << N << "\n\n";

  int *a, *b, *c;
  a = (int*)malloc(N*sizeof (int));
  b = (int*)malloc(N*sizeof (int));
  c = (int*)malloc(N*sizeof (int));
  //cudaMallocHost(&a, N*sizeof(int));
  //cudaMallocHost(&b, N*sizeof(int));
  //cudaMallocHost(&c, N*sizeof(int));

  if(Debug) cout << "a array list:\t";
  rand(a, N);
  if(Debug) cout << "b array list:\t";
  rand(b, N);

  cout << "\n\n@CPU, summing value of size(n) * size(n) ... \n";

  clock_t t;

  // calling cpu
  t=clock();//start time
  cpucal(a,b,c,N);
  t = clock() - t;//total time = end time - start time

  int vmax = 0;
  for (int i = 0; i < N; i++) {
    vmax = (vmax > c[i]) ? vmax : c[i];
  }
  cout << vmax << "\n";

  printf ("CPU Avg time = %lf ms.\n",((((float)t)/CLOCKS_PER_SEC)*1000));

  free(a);
  free(b);
  free(c);

  return 0;
}

Writing cpu-demo.cpp


In [ ]:
%%shell

g++ cpu-demo.cpp -o cpu-demo
./cpu-demo

A GPU is connected.
n:	32768



@CPU, summing value of size(n) * size(n) ... 
6
CPU Avg time = 8229.040039 ms.


GPU Implementation

In [ ]:
%%cuda_group_save -n demo.cu -g share

#include <cstdio>
#include <iostream>
#include <chrono>
#include <thread>
#include <cuda_runtime.h>

#define Debug false


using namespace std;


void checkGPU(){
    const char* gpu_env = std::getenv("COLAB_GPU");
    if (gpu_env && std::atoi(gpu_env) > 0) {
        cout << "A GPU is connected." << endl;
    } else {
        cout << "No accelerator is connected." << endl;
    }
}

void checkGpuMemory() {
    size_t free, total;
    cudaError_t status;
    status = cudaMemGetInfo(&free, &total);
    if (status != cudaSuccess) {
        std::cerr << "Error checking GPU memory: " << cudaGetErrorString(status) << std::endl;
        return;
    }

    // Convert bytes to megabytes
    double freeMB = static_cast<double>(free) / (1024 * 1024);
    double totalMB = static_cast<double>(total) / (1024 * 1024);
    cout << "\n";
    cout << "Free Memory: " << freeMB << " MB" << std::endl;
    cout << "Total Memory: " << totalMB << " MB" << std::endl;
}


void rand(int *a, int n){
  for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		if(Debug) cout << a[i] << "\t";
	}
  if(Debug) cout << "\n";
}


void cpucal (int *a, int *b, int *c, int n, IncrementalRunningAverage *average ) {
  // calculate the average of each of a / each of b
  for (int i=0; i<n; i++){
    for (int j=0; j<n; j++) {
       if(b[j] != 0) average.add(a[i]/b[j]);
       else average.add(0);
    }
    c[i] = average.getAverage();
  }
}


__device__ int getGlobalIdx_2D_2D() {
  int blockId = blockIdx.x + blockIdx.y * gridDim.x;
  int threadId = blockId * (blockDim.x * blockDim.y) + (threadIdx.y * blockDim.x) + threadIdx.x;
  return threadId;
}

__global__ void veccal_kernel (int *a, int *b, int *c, int n, IncrementalRunningAverage *average ) {
  int threadId = getGlobalIdx_2D_2D();
  if (threadId < n && threadId < n) {   // Protect against out of bounds error
    if(b[threadId] != 0) average.add(a[threadId]/b[threadId]);
    else average.add(0);
  }
}


void gpucal (int *a, int *b, int *c, int n ) {
  int *d_a , *d_b , *d_c;
  cudaMalloc(&d_a, n*sizeof(int));
  cudaMalloc(&d_b, n*sizeof(int));
  cudaMalloc(&d_c, n*sizeof(int));
  cudaMemcpy(d_a, a, n*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

  // Define grid and block dimensions
  dim3 block(32, 32);
  dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
  veccal_kernel<<<grid,block>>>(d_a, d_b, d_c, n);

  // Sync Device to Host
  cudaDeviceSynchronize();

  cudaMemcpy(c, d_c, n*sizeof(int), cudaMemcpyDeviceToHost);
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
}


int main(void) {

  checkGPU();
  checkGpuMemory();

  int N=1<<15;
  cout << "n:\t" << N << "\n\n";

  int *a, *b, *c;
  //a = (int*)malloc(N*sizeof (int));
  //b = (int*)malloc(N*sizeof (int));
  //c = (int*)malloc(N*sizeof (int));
  cudaMallocHost(&a, N*sizeof(int));
  cudaMallocHost(&b, N*sizeof(int));
  cudaMallocHost(&c, N*sizeof(int));

  if(Debug) cout << "a array list:\t";
  rand(a, N);
  if(Debug) cout << "b array list:\t";
  rand(b, N);
  clock_t t;


  // calling cpu
  cout << "\n\n@CPU, summing value of size(n) * size(n) ... \n";
  t=clock();//start time
  cpucal(a,b,c,N);
  t = clock() - t;//total time = end time - start time

  int vmax = 0;
  for (int i = 0; i < N; i++) {
    vmax = (vmax > c[i]) ? vmax : c[i];
  }
  cout << vmax << "\n";

  printf ("CPU Avg time = %lf ms.\n",((((float)t)/CLOCKS_PER_SEC)*1000));

  if(false) {
    // calling gpu
    cout << "\n\n@GPU, summing value... \n";

    t=clock();//start time
    gpucal(a,b,c,N);
    t = clock() - t;//total time = end time - start time

    vmax = 0;
    for (int i = 0; i < N; i++) {
      vmax = (vmax > c[i]) ? vmax : c[i];
    }
    cout << vmax << "\n";
    printf ("GPU Avg time = %lf ms.\n",((((float)t)/CLOCKS_PER_SEC)*1000));
  }
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}

In [ ]:
%cuda_group_run -g "share"

A GPU is connected.

Free Memory: 14999.1 MB
Total Memory: 15102.1 MB
n:	32768



@CPU, summing value of size(n) * size(n) ... 
6
CPU Avg time = 8249.514648 ms.

